In [ ]:
# ============================================================
#  TAV v3.0 — Transparent AI Validator
#  Universal AutoML + Per-Model Feature Selection
# ============================================================

import io
import warnings

warnings.filterwarnings ("ignore")

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, roc_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# ─────────────────────────────────────────────
#  CUSTOM CSS
# ─────────────────────────────────────────────
st.set_page_config (
    page_title = "TAV v3.0 · Universal AutoML",
    page_icon = "⚡",
    layout = "wide",
    initial_sidebar_state = "expanded",
)

st.markdown ("""
<style>
@import url('https://fonts.googleapis.com/css2?family=JetBrains+Mono:wght@400;700&family=Syne:wght@400;700;800&display=swap');

html, body, [class*="css"] {
    font-family: 'Syne', sans-serif;
    background: #0a0a0f;
    color: #e2e8f0;
}

.stApp {
    background: linear-gradient(135deg, #0a0a0f 0%, #0d1117 50%, #0a0f1a 100%);
}

.tav-header {
    background: linear-gradient(90deg, #00f5a0 0%, #00d9f5 50%, #7c3aed 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-family: 'Syne', sans-serif;
    font-size: 3.2rem;
    font-weight: 800;
    letter-spacing: -1px;
    margin-bottom: 0;
}

.tav-sub {
    color: #64748b;
    font-family: 'JetBrains Mono', monospace;
    font-size: 0.85rem;
    letter-spacing: 3px;
    text-transform: uppercase;
    margin-bottom: 2rem;
}

.section-title {
    font-family: 'JetBrains Mono', monospace;
    font-size: 0.7rem;
    letter-spacing: 4px;
    text-transform: uppercase;
    color: #00f5a0;
    margin-bottom: 0.5rem;
    padding-top: 2rem;
}

.best-model-banner {
    background: linear-gradient(90deg, rgba(0,245,160,0.15), rgba(0,217,245,0.15));
    border: 1px solid rgba(0,245,160,0.4);
    border-radius: 12px;
    padding: 1.2rem 2rem;
    font-family: 'Syne', sans-serif;
    font-size: 1.2rem;
    font-weight: 700;
    color: #00f5a0;
    text-align: center;
    margin: 1rem 0;
}

.stButton > button {
    background: linear-gradient(90deg, #00f5a0, #00d9f5);
    color: #0a0a0f;
    font-family: 'Syne', sans-serif;
    font-weight: 700;
    border: none;
    border-radius: 8px;
    padding: 0.6rem 2rem;
    font-size: 0.95rem;
    letter-spacing: 1px;
    width: 100%;
}

.stButton > button:hover {
    opacity: 0.85;
    transform: translateY(-1px);
}

.stDownloadButton > button {
    background: rgba(124,58,237,0.2);
    border: 1px solid rgba(124,58,237,0.6);
    color: #a78bfa;
    font-family: 'JetBrains Mono', monospace;
    border-radius: 8px;
    width: 100%;
}

div[data-testid="stDataFrame"] {
    border-radius: 10px;
    overflow: hidden;
}

.stSelectbox label, .stMultiSelect label, .stFileUploader label {
    color: #94a3b8;
    font-size: 0.85rem;
    font-family: 'JetBrains Mono', monospace;
    letter-spacing: 1px;
    text-transform: uppercase;
}

section[data-testid="stSidebar"] {
    background: rgba(255,255,255,0.02);
    border-right: 1px solid rgba(255,255,255,0.06);
}

.sidebar-logo {
    font-family: 'Syne', sans-serif;
    font-size: 1.8rem;
    font-weight: 800;
    background: linear-gradient(90deg, #00f5a0, #00d9f5);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}

hr {
    border-color: rgba(255,255,255,0.07);
}

.stAlert {
    background: rgba(0,245,160,0.07);
    border-color: rgba(0,245,160,0.3);
    border-radius: 10px;
}
</style>
""", unsafe_allow_html = True)

# ─────────────────────────────────────────────
#  SIDEBAR
# ─────────────────────────────────────────────
with st.sidebar:
    st.markdown ('<div class="sidebar-logo">⚡ TAV v3.0</div>', unsafe_allow_html = True)
    st.markdown ('<div style="color:#64748b;font-size:0.7rem;letter-spacing:3px;">UNIVERSAL AUTOML</div>',
                 unsafe_allow_html = True)
    st.markdown ("---")

    st.markdown ("**⚙️ Configuration**")
    test_size = st.slider ("Test Split Size", 0.10, 0.40, 0.20, 0.05)
    cv_folds = st.slider ("Cross-Validation Folds", 3, 10, 5)
    apply_smote = st.toggle ("Apply SMOTE (Imbalance Fix)", value = True)

    st.markdown ("---")
    st.markdown ("**🎯 Feature Selection**")
    enable_feature_selection = st.toggle ("Enable Per-Model Feature Selection", value = True)
    if enable_feature_selection:
        feature_selection_method = st.selectbox (
            "Selection Method",
            ["Auto (Best for Each Model)", "Mutual Information", "F-Statistic", "Chi-Square"]
        )
        n_features_pct = st.slider ("% Features to Keep", 20, 100, 70, 10)

    st.markdown ("---")
    st.markdown ("**🤖 Models**")
    model_choices = st.multiselect (
        "Select Models to Train",
        ["Logistic Regression", "Decision Tree", "Random Forest",
         "Gradient Boosting", "XGBoost"],
        default = ["Logistic Regression", "Decision Tree", "Random Forest",
                   "Gradient Boosting", "XGBoost"]
    )

    st.markdown ("---")
    st.markdown (
        '<div style="color:#334155;font-size:0.72rem;font-family:monospace;">v3.0 · Universal Classification</div>',
        unsafe_allow_html = True)

# ─────────────────────────────────────────────
#  HEADER
# ─────────────────────────────────────────────
st.markdown ('<div class="tav-header">TAV v3.0</div>', unsafe_allow_html = True)
st.markdown ('<div class="tav-sub">Universal AutoML + Intelligent Feature Selection</div>', unsafe_allow_html = True)

# ─────────────────────────────────────────────
#  FILE UPLOAD
# ─────────────────────────────────────────────
uploaded_file = st.file_uploader ("Upload your CSV dataset", type = ["csv"])

if uploaded_file is None:
    st.info ("👆 Upload a CSV file to begin. TAV handles numeric, categorical, mixed, and text datasets.")
    st.stop ()

# ─────────────────────────────────────────────
#  LOAD DATA
# ─────────────────────────────────────────────
df_raw = pd.read_csv (uploaded_file)

# ── Dataset Preview ──────────────────────────
st.markdown ('<div class="section-title">01 · Dataset Overview</div>', unsafe_allow_html = True)

c1, c2, c3, c4 = st.columns (4)
c1.metric ("Rows", f"{df_raw.shape [0]:,}")
c2.metric ("Columns", df_raw.shape [1])
c3.metric ("Numeric", len (df_raw.select_dtypes (include = np.number).columns))
c4.metric ("Missing %", f"{df_raw.isnull ().mean ().mean () * 100:.1f}%")

st.dataframe (df_raw.head (8), use_container_width = True)

# ─────────────────────────────────────────────
#  TARGET SELECTION
# ─────────────────────────────────────────────
st.markdown ('<div class="section-title">02 · Target Column</div>', unsafe_allow_html = True)
target_col = st.selectbox ("Select the TARGET (label) column", df_raw.columns)

if st.button ("⚡ Run TAV AutoML"):

    df = df_raw.copy ()

    # ─────────────────────────────────────────
    #  EDA SECTION
    # ─────────────────────────────────────────
    st.markdown ('<div class="section-title">03 · Exploratory Data Analysis</div>', unsafe_allow_html = True)

    tab1, tab2, tab3, tab4 = st.tabs (["📊 Class Distribution", "🔥 Correlation Heatmap",
                                       "📉 Missing Values", "📈 Feature Stats"])

    with tab1:
        fig, ax = plt.subplots (figsize = (6, 3.5), facecolor = '#0d1117')
        ax.set_facecolor ('#0d1117')
        counts = df [target_col].value_counts ()
        colors = plt.cm.viridis (np.linspace (0.2, 0.8, len (counts)))
        bars = ax.bar (counts.index.astype (str), counts.values, color = colors, width = 0.5, edgecolor = 'none')
        for bar, val in zip (bars, counts.values):
            ax.text (bar.get_x () + bar.get_width () / 2, bar.get_height () + max (counts) * 0.01,
                     f'{val:,}', ha = 'center', color = '#e2e8f0', fontsize = 10, fontweight = 'bold')
        ax.set_xlabel ('Class', color = '#64748b')
        ax.set_ylabel ('Count', color = '#64748b')
        ax.set_title ('Target Class Distribution', color = '#e2e8f0', pad = 12)
        ax.tick_params (colors = '#64748b')
        for spine in ax.spines.values ():
            spine.set_edgecolor ('#1e293b')
        st.pyplot (fig, use_container_width = True)

    with tab2:
        num_df = df.select_dtypes (include = np.number)
        if len (num_df.columns) > 1:
            fig, ax = plt.subplots (figsize = (max (6, len (num_df.columns)), max (5, len (num_df.columns) - 1)),
                                    facecolor = '#0d1117')
            ax.set_facecolor ('#0d1117')
            corr = num_df.corr ()
            mask = np.triu (np.ones_like (corr, dtype = bool))
            cmap = sns.diverging_palette (150, 275, s = 80, l = 55, as_cmap = True)
            sns.heatmap (corr, mask = mask, cmap = cmap, center = 0, linewidths = 0.5,
                         linecolor = '#0a0a0f', annot = True, fmt = '.2f',
                         annot_kws = {'size': 8, 'color': '#e2e8f0'}, ax = ax,
                         cbar_kws = {'shrink': 0.7})
            ax.tick_params (colors = '#94a3b8', labelsize = 8)
            ax.set_title ("Correlation Matrix (Numeric Features)", color = '#e2e8f0', pad = 12)
            st.pyplot (fig, use_container_width = True)
        else:
            st.info ("Not enough numeric columns for correlation heatmap.")

    with tab3:
        missing = df.isnull ().sum ()
        missing = missing [missing > 0]
        if len (missing) == 0:
            st.success ("✅ No missing values found!")
        else:
            fig, ax = plt.subplots (figsize = (7, max (3, len (missing) * 0.5)), facecolor = '#0d1117')
            ax.set_facecolor ('#0d1117')
            pct = (missing / len (df) * 100).sort_values (ascending = True)
            colors_m = ['#ef4444' if p > 30 else '#f59e0b' if p > 10 else '#00f5a0' for p in pct.values]
            ax.barh (pct.index, pct.values, color = colors_m, edgecolor = 'none', height = 0.6)
            ax.set_xlabel ('Missing %', color = '#64748b')
            ax.set_title ('Missing Values by Feature', color = '#e2e8f0', pad = 12)
            ax.tick_params (colors = '#94a3b8', labelsize = 9)
            for spine in ax.spines.values ():
                spine.set_edgecolor ('#1e293b')
            st.pyplot (fig, use_container_width = True)

    with tab4:
        num_cols_desc = df.select_dtypes (include = np.number).columns
        if len (num_cols_desc) > 0:
            st.markdown ("**Numeric Features:**")
            num_desc = df [num_cols_desc].describe ().T
            st.dataframe (num_desc.style.format ("{:.3f}"), use_container_width = True)

        cat_cols_eda = df.select_dtypes (exclude = np.number).columns
        if len (cat_cols_eda) > 0:
            st.markdown ("**Categorical Features:**")
            for col in cat_cols_eda:
                if col != target_col:
                    with st.expander (f"📋 {col}"):
                        vc = df [col].value_counts ().head (10)
                        st.bar_chart (vc)

        if len (num_cols_desc) == 0 and len (cat_cols_eda) == 0:
            st.info ("No features to display (only target column).")

    # ─────────────────────────────────────────
    #  PREPROCESSING PIPELINE
    # ─────────────────────────────────────────
    st.markdown ('<div class="section-title">04 · Smart Preprocessing</div>', unsafe_allow_html = True)

    X = df.drop (columns = [target_col])
    y = df [target_col].copy ()

    # ── Fill missing target values with mode ──
    if y.isnull ().any ():
        fill_val = y.mode () [0]
        y = y.fillna (fill_val)
        st.info (f"ℹ️  Missing target filled with: '{fill_val}'")

    # ── LabelEncoder for target ──
    target_le = LabelEncoder ()
    y = pd.Series (target_le.fit_transform (y.astype (str)), name = target_col)
    class_mapping = {cls: int (idx) for idx, cls in enumerate (target_le.classes_)}
    st.info (f"🏷️  Target encoded → {class_mapping}")

    # Identify column types
    num_cols = X.select_dtypes (include = np.number).columns.tolist ()
    cat_cols = X.select_dtypes (include = 'object').columns.tolist ()

    # Detect binary vs nominal
    binary_cat = [c for c in cat_cols if X [c].nunique () == 2]
    nominal_cat = [c for c in cat_cols if X [c].nunique () > 2]

    st.markdown (f"""
    | Type | Count | Columns |
    |------|-------|---------|
    | **Numeric** | {len (num_cols)} | {', '.join (num_cols [:5]) if num_cols else '—'}{'...' if len (num_cols) > 5 else ''} |
    | **Binary Cat** | {len (binary_cat)} | {', '.join (binary_cat [:5]) if binary_cat else '—'}{'...' if len (binary_cat) > 5 else ''} |
    | **Nominal Cat** | {len (nominal_cat)} | {', '.join (nominal_cat [:5]) if nominal_cat else '—'}{'...' if len (nominal_cat) > 5 else ''} |
    """)

    # Numeric pipeline: median impute + scale
    numeric_pipe = Pipeline ([
        ('imputer', SimpleImputer (strategy = 'median')),
        ('scaler', StandardScaler ())
    ])

    # Binary cat: mode impute + ordinal (0/1)
    binary_pipe = Pipeline ([
        ('imputer', SimpleImputer (strategy = 'most_frequent')),
        ('encoder', OrdinalEncoder (handle_unknown = 'use_encoded_value', unknown_value = -1))
    ])

    # Nominal cat: mode impute + one-hot
    from sklearn.preprocessing import OneHotEncoder

    nominal_pipe = Pipeline ([
        ('imputer', SimpleImputer (strategy = 'most_frequent')),
        ('encoder', OneHotEncoder (handle_unknown = 'ignore', sparse_output = False))
    ])

    transformers = []
    if num_cols:    transformers.append (('num', numeric_pipe, num_cols))
    if binary_cat:  transformers.append (('bin', binary_pipe, binary_cat))
    if nominal_cat: transformers.append (('nom', nominal_pipe, nominal_cat))

    preprocessor = ColumnTransformer (transformers = transformers, remainder = 'drop')

    # ── CRITICAL: Split BEFORE preprocessing ──
    # Check if stratification is safe
    class_counts = y.value_counts ()
    min_class_size = class_counts.min ()
    can_stratify = min_class_size >= 2  # Need at least 2 samples per class

    if not can_stratify:
        st.warning (
            f"⚠️  Dataset too small for stratified split ({min_class_size} samples in smallest class). Using random split.")
        stratify_param = None
    else:
        stratify_param = y

    X_train_raw, X_test_raw, y_train, y_test = train_test_split (
        X, y, test_size = test_size, random_state = 42, stratify = stratify_param
    )

    # Fit preprocessor on training data only
    X_train = preprocessor.fit_transform (X_train_raw)
    X_test = preprocessor.transform (X_test_raw)

    st.success (f"✅ Preprocessing done. Training: {X_train.shape [0]} rows × {X_train.shape [1]} features")

    # SMOTE
    if apply_smote and can_stratify:  # SMOTE also needs at least 2 samples per class
        class_counts_train = np.bincount (y_train.values)
        imbalance_ratio = class_counts_train.min () / class_counts_train.max ()
        if imbalance_ratio < 0.8 and class_counts_train.min () >= 2:
            try:
                sm = SMOTE (random_state = 42)
                X_train, y_train = sm.fit_resample (X_train, y_train)
                st.info (f"⚖️  SMOTE applied. Balanced training set: {X_train.shape [0]:,} samples")
            except ValueError as e:
                st.warning (f"⚠️  SMOTE skipped: {str (e)}")
        else:
            st.info ("📊 Dataset reasonably balanced or too small for SMOTE.")
    elif apply_smote and not can_stratify:
        st.warning ("⚠️  SMOTE disabled: dataset too small (classes with only 1 sample).")

    # ─────────────────────────────────────────
    #  PER-MODEL FEATURE SELECTION
    # ─────────────────────────────────────────
    if enable_feature_selection:
        st.markdown ('<div class="section-title">05 · Intelligent Feature Selection</div>', unsafe_allow_html = True)

        n_features_to_keep = max (5, int (X_train.shape [1] * n_features_pct / 100))

        # Define per-model selection strategies
        selection_strategies = {
            "Logistic Regression": "f_classif",  # Linear models like F-statistic
            "Decision Tree": "mutual_info",  # Tree models like mutual info
            "Random Forest": "mutual_info",
            "Gradient Boosting": "mutual_info",
            "XGBoost": "mutual_info"
        }

        if feature_selection_method != "Auto (Best for Each Model)":
            # Override with user choice
            method_map = {
                "Mutual Information": "mutual_info",
                "F-Statistic": "f_classif",
                "Chi-Square": "chi2"
            }
            for k in selection_strategies:
                selection_strategies [k] = method_map [feature_selection_method]

        st.info (f"🎯 Selecting top {n_features_pct}% features ({n_features_to_keep} features) per model")

        feature_selectors = {}
        for model_name in model_choices:
            strategy = selection_strategies.get (model_name, "mutual_info")

            if strategy == "f_classif":
                selector = SelectKBest (f_classif, k = min (n_features_to_keep, X_train.shape [1]))
            elif strategy == "chi2":
                # Chi2 requires non-negative features
                selector = SelectKBest (chi2, k = min (n_features_to_keep, X_train.shape [1]))
            else:  # mutual_info
                selector = SelectKBest (mutual_info_classif, k = min (n_features_to_keep, X_train.shape [1]))

            feature_selectors [model_name] = selector

        st.success (f"✅ Feature selectors initialized for {len (model_choices)} models")

    # ─────────────────────────────────────────
    #  MODEL TRAINING
    # ─────────────────────────────────────────
    st.markdown ('<div class="section-title">06 · Model Training & Benchmarking</div>', unsafe_allow_html = True)

    ALL_MODELS = {
        "Logistic Regression": LogisticRegression (max_iter = 1000, random_state = 42),
        "Decision Tree": DecisionTreeClassifier (random_state = 42),
        "Random Forest": RandomForestClassifier (n_estimators = 100, random_state = 42),
        "Gradient Boosting": GradientBoostingClassifier (n_estimators = 100, random_state = 42),
        "XGBoost": XGBClassifier (n_estimators = 100, use_label_encoder = False,
                                  eval_metric = 'logloss', random_state = 42, verbosity = 0),
    }

    selected_models = {k: v for k, v in ALL_MODELS.items () if k in model_choices}

    # Auto-detect binary vs multiclass
    n_classes = len (np.unique (y))
    is_binary = (n_classes == 2)
    avg = 'binary' if is_binary else 'weighted'
    cv_scoring = 'roc_auc' if is_binary else 'roc_auc_ovr_weighted'

    if not is_binary:
        st.info (f"🔢 Multiclass detected ({n_classes} classes) — using weighted averaging")

    results = []
    trained = {}
    feature_importance_data = {}

    prog = st.progress (0, text = "Training models…")

    # Adjust CV folds for small datasets
    min_samples_per_fold = X_train.shape [0] // cv_folds
    if min_samples_per_fold < n_classes:
        safe_cv_folds = max (2, X_train.shape [0] // n_classes)
        st.warning (f"⚠️  Reducing CV folds from {cv_folds} to {safe_cv_folds} (dataset too small)")
        cv_folds_actual = safe_cv_folds
    else:
        cv_folds_actual = cv_folds

    skf = StratifiedKFold (n_splits = cv_folds_actual, shuffle = True, random_state = 42) if can_stratify else None

    for i, (name, model) in enumerate (selected_models.items ()):
        prog.progress (i / len (selected_models), text = f"Training {name}…")

        # Apply feature selection if enabled
        if enable_feature_selection:
            selector = feature_selectors [name]
            X_train_selected = selector.fit_transform (X_train, y_train)
            X_test_selected = selector.transform (X_test)
            selected_features = selector.get_support ()
            n_selected = selected_features.sum ()
        else:
            X_train_selected = X_train
            X_test_selected = X_test
            n_selected = X_train.shape [1]

        # Train model
        model.fit (X_train_selected, y_train)
        y_pred = model.predict (X_test_selected)
        y_proba = model.predict_proba (X_test_selected)

        # ROC-AUC calculation
        if is_binary:
            y_prob_for_auc = y_proba [:, 1]
            try:
                auc = roc_auc_score (y_test, y_prob_for_auc)
            except ValueError as e:
                st.warning (f"ROC-AUC calculation failed for {name}: {e}")
                auc = np.nan
        else:
            y_prob_for_auc = y_proba
            # Check if test set has all classes
            unique_train_classes = np.unique (y_train)
            unique_test_classes = np.unique (y_test)

            if len (unique_test_classes) < len (unique_train_classes):
                # Test set missing some classes - can't compute proper OVR AUC
                st.warning (f"⚠️  {name}: Test set missing some classes. Using accuracy instead of AUC.")
                auc = accuracy_score (y_test, y_pred)  # Fallback to accuracy
            else:
                try:
                    auc = roc_auc_score (y_test, y_prob_for_auc,
                                         multi_class = 'ovr', average = 'weighted')
                except ValueError as e:
                    st.warning (f"⚠️  {name}: ROC-AUC failed ({str (e)}). Using accuracy.")
                    auc = accuracy_score (y_test, y_pred)

        # Cross-validation
        if skf is not None:
            try:
                cv_scores = cross_val_score (model, X_train_selected, y_train,
                                             cv = skf, scoring = cv_scoring, n_jobs = -1)
                cv_mean, cv_std = cv_scores.mean (), cv_scores.std ()
            except ValueError:
                # CV failed (too small), skip
                cv_mean, cv_std = np.nan, np.nan
        else:
            # Dataset too small for CV
            cv_mean, cv_std = np.nan, np.nan

        results.append ({
            "Model": name,
            "Features": n_selected,
            "Accuracy": accuracy_score (y_test, y_pred),
            "Precision": precision_score (y_test, y_pred, average = avg, zero_division = 0),
            "Recall": recall_score (y_test, y_pred, average = avg, zero_division = 0),
            "F1 Score": f1_score (y_test, y_pred, average = avg, zero_division = 0),
            "ROC-AUC": auc,
            "CV AUC μ": cv_mean,
            "CV AUC σ": cv_std,
        })

        if enable_feature_selection:
            trained [name] = (model, y_proba, y_prob_for_auc, selector, selected_features)
            feature_importance_data [name] = selected_features
        else:
            trained [name] = (model, y_proba, y_prob_for_auc, None, None)

    prog.progress (1.0, text = "✅ All models trained!")

    results_df = pd.DataFrame (results).sort_values ("ROC-AUC", ascending = False).reset_index (drop = True)


    # Style results table
    def style_table (df):
        return df.style.format ({
            "Features": "{:d}",
            "Accuracy": "{:.4f}", "Precision": "{:.4f}", "Recall": "{:.4f}",
            "F1 Score": "{:.4f}", "ROC-AUC": "{:.4f}",
            "CV AUC μ": "{:.4f}", "CV AUC σ": "{:.4f}",
        }).background_gradient (subset = ["ROC-AUC", "F1 Score"], cmap = "Greens")


    st.dataframe (style_table (results_df), use_container_width = True, hide_index = True)

    # ── Feature Selection Comparison ─────────
    if enable_feature_selection:
        st.markdown ('<div class="section-title">07 · Feature Selection Analysis</div>', unsafe_allow_html = True)

        fig, ax = plt.subplots (figsize = (10, 4), facecolor = '#0d1117')
        ax.set_facecolor ('#0d1117')

        models_list = list (feature_importance_data.keys ())
        feature_counts = [feature_importance_data [m].sum () for m in models_list]

        colors_feat = ['#00f5a0', '#00d9f5', '#7c3aed', '#f59e0b', '#ef4444']
        bars = ax.barh (models_list, feature_counts, color = colors_feat [:len (models_list)],
                        height = 0.6, edgecolor = 'none')

        for bar, val in zip (bars, feature_counts):
            ax.text (bar.get_width () + 1, bar.get_y () + bar.get_height () / 2,
                     f'{val}', va = 'center', color = '#e2e8f0', fontsize = 10, fontweight = 'bold')

        ax.set_xlabel ('Number of Features Selected', color = '#64748b')
        ax.set_title ('Features Selected Per Model', color = '#e2e8f0', pad = 12)
        ax.tick_params (colors = '#94a3b8')
        for spine in ax.spines.values ():
            spine.set_edgecolor ('#1e293b')
        st.pyplot (fig, use_container_width = True)

    # ── Metric Comparison Charts ─────────────
    st.markdown ('<div class="section-title">08 · Metric Comparison</div>', unsafe_allow_html = True)

    metrics_to_plot = ["Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"]
    fig, axes = plt.subplots (1, 5, figsize = (18, 4), facecolor = '#0d1117')
    palette = ['#00f5a0', '#00d9f5', '#7c3aed', '#f59e0b', '#ef4444']

    for ax, metric, color in zip (axes, metrics_to_plot, palette):
        ax.set_facecolor ('#0d1117')
        bars = ax.barh (results_df ["Model"], results_df [metric], color = color, alpha = 0.85,
                        edgecolor = 'none', height = 0.55)
        for bar, val in zip (bars, results_df [metric]):
            ax.text (bar.get_width () - 0.01, bar.get_y () + bar.get_height () / 2,
                     f'{val:.3f}', va = 'center', ha = 'right',
                     color = '#0a0a0f', fontsize = 8.5, fontweight = 'bold')
        ax.set_xlabel ('Score', color = '#64748b', fontsize = 8)
        ax.set_title (metric, color = '#e2e8f0', fontsize = 10, pad = 8)
        ax.tick_params (colors = '#94a3b8', labelsize = 8)
        ax.set_xlim (0, 1.05)
        for spine in ax.spines.values ():
            spine.set_edgecolor ('#1e293b')

    plt.tight_layout ()
    st.pyplot (fig, use_container_width = True)

    # ── ROC Curves ────────────
    st.markdown ('<div class="section-title">09 · ROC Curve Comparison</div>', unsafe_allow_html = True)

    fig, ax = plt.subplots (figsize = (8, 5), facecolor = '#0d1117')
    ax.set_facecolor ('#0d1117')

    roc_colors = ['#00f5a0', '#00d9f5', '#7c3aed', '#f59e0b', '#ef4444']
    for (name, data), color in zip (trained.items (), roc_colors):
        model, y_proba, y_prob_for_auc = data [0], data [1], data [2]

        if is_binary:
            fpr, tpr, _ = roc_curve (y_test, y_prob_for_auc)
            auc_val = roc_auc_score (y_test, y_prob_for_auc)
            ax.plot (fpr, tpr, label = f'{name} (AUC={auc_val:.3f})', color = color, lw = 2)
        else:
            from sklearn.preprocessing import label_binarize

            classes = np.unique (y)
            y_test_bin = label_binarize (y_test, classes = classes)
            auc_val = roc_auc_score (y_test, y_prob_for_auc, multi_class = 'ovr', average = 'weighted')
            fpr, tpr, _ = roc_curve (y_test_bin.ravel (), y_prob_for_auc.ravel ())
            ax.plot (fpr, tpr, label = f'{name} (AUC={auc_val:.3f})', color = color, lw = 2)

    ax.plot ([0, 1], [0, 1], color = '#334155', lw = 1.5, linestyle = '--', label = 'Random')
    ax.fill_between ([0, 1], [0, 1], alpha = 0.05, color = '#334155')
    ax.set_xlabel ('False Positive Rate', color = '#64748b')
    ax.set_ylabel ('True Positive Rate', color = '#64748b')
    ax.set_title ('ROC Curve — All Models', color = '#e2e8f0', pad = 12)
    ax.legend (loc = 'lower right', fontsize = 8.5, framealpha = 0.15,
               labelcolor = '#e2e8f0', facecolor = '#0d1117', edgecolor = '#1e293b')
    ax.tick_params (colors = '#94a3b8')
    for spine in ax.spines.values ():
        spine.set_edgecolor ('#1e293b')
    st.pyplot (fig, use_container_width = False)

    # ─────────────────────────────────────────
    #  BEST MODEL
    # ─────────────────────────────────────────
    best_name = results_df.iloc [0] ["Model"]
    best_model_data = trained [best_name]
    best_model = best_model_data [0]
    best_selector = best_model_data [3] if enable_feature_selection else None
    best_auc = results_df.iloc [0] ["ROC-AUC"]
    best_f1 = results_df.iloc [0] ["F1 Score"]

    st.markdown (
        f'<div class="best-model-banner">🏆 Best Model: {best_name} &nbsp;|&nbsp; AUC = {best_auc:.4f} &nbsp;|&nbsp; F1 = {best_f1:.4f}</div>',
        unsafe_allow_html = True
    )

    # ── Confusion Matrix ─────────────────────
    st.markdown ('<div class="section-title">10 · Confusion Matrix (Best Model)</div>', unsafe_allow_html = True)

    if enable_feature_selection and best_selector:
        X_test_best = best_selector.transform (X_test)
    else:
        X_test_best = X_test

    y_pred_best = best_model.predict (X_test_best)
    cm = confusion_matrix (y_test, y_pred_best)

    fig_size = (4.5, 3.5) if is_binary else (max (5, n_classes * 1.5), max (4, n_classes * 1.2))
    fig, ax = plt.subplots (figsize = fig_size, facecolor = '#0d1117')
    ax.set_facecolor ('#0d1117')

    cmap_custom = sns.diverging_palette (150, 275, s = 80, l = 55, as_cmap = True)
    class_labels = [str (c) for c in target_le.classes_]

    sns.heatmap (cm, annot = True, fmt = 'd', cmap = cmap_custom,
                 linewidths = 2, linecolor = '#0a0a0f',
                 annot_kws = {'size': 14 if is_binary else 11, 'weight': 'bold', 'color': '#e2e8f0'},
                 cbar = False, ax = ax,
                 xticklabels = [f'Pred {lbl}' for lbl in class_labels],
                 yticklabels = [f'True {lbl}' for lbl in class_labels])
    ax.tick_params (colors = '#94a3b8', labelsize = 9)
    ax.set_title (f'Confusion Matrix — {best_name}', color = '#e2e8f0', pad = 12)

    if is_binary:
        tn, fp, fn, tp = cm.ravel ()
        col1, col2 = st.columns ([1, 2])
        with col1:
            st.pyplot (fig)
        with col2:
            st.markdown (f"""
            | Metric | Value |
            |--------|-------|
            | True Positives (TP) | **{tp}** |
            | True Negatives (TN) | **{tn}** |
            | False Positives (FP) | **{fp}** |
            | False Negatives (FN) | **{fn}** |
            | Sensitivity | **{tp / (tp + fn) if (tp + fn) > 0 else 0:.4f}** |
            | Specificity | **{tn / (tn + fp) if (tn + fp) > 0 else 0:.4f}** |
            | Precision | **{tp / (tp + fp) if (tp + fp) > 0 else 0:.4f}** |
            """)
    else:
        st.pyplot (fig)
        class_acc = cm.diagonal () / cm.sum (axis = 1)
        acc_df = pd.DataFrame ({
            'Class': class_labels,
            'Samples': cm.sum (axis = 1),
            'Correct': cm.diagonal (),
            'Accuracy': class_acc
        })
        st.markdown ("**Per-Class Performance:**")
        st.dataframe (acc_df.style.format ({'Accuracy': '{:.2%}'}), use_container_width = True)

    # ─────────────────────────────────────────
    #  EXPORT
    # ─────────────────────────────────────────
    st.markdown ('<div class="section-title">11 · Export</div>', unsafe_allow_html = True)

    col_d1, col_d2 = st.columns (2)

    with col_d1:
        model_bytes = io.BytesIO ()
        export_data = {
            'model': best_model,
            'preprocessor': preprocessor,
            'target_encoder': target_le,
            'feature_selector': best_selector,
            'feature_names': preprocessor.get_feature_names_out () if hasattr (preprocessor,
                                                                               'get_feature_names_out') else None
        }
        joblib.dump (export_data, model_bytes)
        model_bytes.seek (0)
        st.download_button (
            label = "⬇ Download Best Model (.pkl)",
            data = model_bytes,
            file_name = f"TAV_v3_{best_name.replace (' ', '_')}_model.pkl",
            mime = "application/octet-stream"
        )

    with col_d2:
        csv_bytes = results_df.to_csv (index = False).encode ()
        st.download_button (
            label = "⬇ Download Metrics Report (.csv)",
            data = csv_bytes,
            file_name = "TAV_v3_model_comparison.csv",
            mime = "text/csv"
        )

    st.markdown ("---")
    st.markdown (
        '<div style="text-align:center;color:#334155;font-family:monospace;font-size:0.75rem;">'
        'TAV v3.0 · Universal AutoML · Built with Streamlit + Scikit-learn'
        '</div>',
        unsafe_allow_html = True
    )